# Trabalho 1 de Segurança Computacional


O trabalho e o código estão disponíveis neste [repositório](https://github.com/marqueswill/seguranca)

Identificação:
- *Aluno:* Willyan Marques de Melo 
- *Matrícula:* 221020940
- *Professor(a):* Lorena de Soza Bezerra Borges
- *Turma:* CIC0201 - Turma 2


## Introdução

O presente trabalho apresenta a cifra Data Encryption Standard (DES) e a implementação da sua versão simplificada (Simplified DES ou S-DES) como forma de exemplificação do algoritmo original. 

### O que é o DES?
   
O DES é um algoritmo clássico de criptografia foi criado pelo atual NIST para o governo dos EUA na década de 1970 como um novo padrão de criptografia proposto pela IBM. Esse algoritmo foi feito com base na Cifra de Feistel e foi amplamente utilizados nas décadas seguintes, até se tornar obsoleto por conta dos avanços tecnológicos e de criptoanálise. O DES foi quebrado em 1999 e, embora outras versões desse algortimo tenham surgido como alternativa de uso, como o 3DES (triple DES), por conta disso entrou em desuso em favor de cifras mais modernas e seguranças como o Advanced Encryption Algorithm (AES).

### A Cifra de Feistel
   
A essência do DES é a cifra de Feistel, que consiste em 16 rodadas de funções de permutação e substituição para incrementar a confusão e a difusão do ciphertext. Cada rodada possui sua própria subchave gerada a partir da chave inicial do algoritmo. Além disso, a cada iteração as metades são trocadas e embaralhas utilizando o resultado do round anterior.

### Overview do DES

O DES é subdivido em 4 partes principais:
   1. Permutação inicial
   2. Rodadas de Feistel
   3. Swap dos bits
   4. Permutação Final

Cada parte do algoritmo principal e da cifra de Feistel serão explicados mais profundamente nas próximas seções


## Funções Auxiliares
O algotimo apresentado no relatório foi implementado utilizando-se a linguagem Python e, embora o python tenha suporte para valores binários, durante o desenvolvimento das cifras surgiu-se a necessidade de padronizar a representação dos valores em binário do código para que pudessemos manter um controle mais preciso do número de bits. Além disso, aqui também foram adicionadas funções auxiliares que precisaram ser implementadas ao decorrer do projeto, já que nenhuma biblioteca externa foi utilizada.

In [402]:
from typing import List


# Representa um número binário com tamanho fixo
class Bin:
    def __init__(self, value: int, size: int):
        self.value = value  # Valor inteiro
        self.size = size  # Tamanho em bits

    # Representação para debug (em hexadecimal)
    def __repr__(self):
        return f"Bin(value={self.to_hex()}, size={self.size})"

    # Representação em string (em hexadecimal)
    def __str__(self):
        return self.to_hex()

    # Verifica igualdade com outro objeto Bin
    def __eq__(self, other):
        if isinstance(other, Bin):
            return self.value == other.value and self.size == other.size
        return False

    # Retorna representação binária com zeros à esquerda
    def to_bin(self, group_size=0):
        raw_bin = f"{self.value:0{self.size}b}"
        if group_size <= 0:
            return f"0b{raw_bin}"
        else:
            grouped = "_".join(
                raw_bin[i : i + group_size] for i in range(0, len(raw_bin), group_size)
            )
            return f"0b{grouped}"

    # Retorna representação hexadecimal com zeros à esquerda
    def to_hex(self):
        hex_digits = (self.size + 3) // 4
        return f"0x{self.value:0{hex_digits}X}"


# Extrai bits com base em uma tabela de posições e retorna como novo Bin
def extract_bits(bits: Bin, table: List[List[int]]) -> Bin:
    extracted_bits = []
    for row in table:
        for bit_pos in row:
            bit = (bits.value >> (bits.size - bit_pos)) & 1
            extracted_bits.append(bit)
    result = 0
    for bit in extracted_bits:
        result = (result << 1) | bit
    return Bin(result, len(extracted_bits))


# Divide o valor Bin em duas metades e retorna como dois objetos Bin
def halve(bits: Bin):
    half = (bits.size + 1) // 2
    left = (bits.value >> half) & ((1 << half) - 1)
    right = bits.value & ((1 << half) - 1)
    return Bin(left, half), Bin(right, half)


# Divide o Bin em pedaços menores de tamanho fixo
def split_bits(bits: Bin, chunk_size: int):
    chunks: List[Bin] = []
    for i in range(0, bits.size, chunk_size):
        chunk_value = (bits.value >> (bits.size - i - chunk_size)) & (
            (1 << chunk_size) - 1
        )
        chunks.append(Bin(chunk_value, chunk_size))
    return chunks


# Junta vários objetos Bin em um só
def fuse_bits(*bins: Bin):
    total_value = 0
    total_size = 0
    for b in bins:
        total_value = (total_value << b.size) | b.value
        total_size += b.size
    return Bin(total_value, total_size)


# Troca as metades esquerda e direita de um Bin
def swap_halves(bits: Bin):
    left, right = halve(bits)
    return fuse_bits(right, left)


# Realiza rotação circular para a esquerda
def circular_left_shift(bits: Bin, shift_value: int):
    shift_value %= bits.size
    discarded = bits.value >> (bits.size - shift_value)
    mask = (1 << bits.size) - 1
    shifted = bits.value << shift_value & mask
    return Bin(shifted | discarded, bits.size)


# Realiza operação XOR bit a bit entre dois Bin
def xor(bits1: Bin, bits2: Bin):
    return Bin(bits1.value ^ bits2.value, bits1.size)

In [403]:
class Logger:
    def __init__(self):
        self.logs = {}

    def log(self, id:str,message: str):
        self.logs[id] = message

    def get_logs(self, id: str):
        if id in self.logs.keys():
            return "\n".join(self.logs[id])
        else:
            return "No logs found for this id."

## Caixas de Permutação 
Abaixo segueem as tabelas de permutação utilizadas para o DES (initial_perm, final_perm), para geração das subchaves (per1, per2, shift_table) e para as rodadas de feistel (dbox, pbox, sbox).
Essas tabelas são utilizadas como entrada para a função `extract_bits`, que extrai os bits das posições indicadas nas tabelas e retorn um valor binário deles.

In [404]:
# Initial permutation
initial_perm = [
    [2, 6, 3, 1, 4, 8, 5, 7],
]

# Final permutation
final_perm = [
    [4, 1, 3, 5, 7, 2, 8, 6],
]

# Permutade choice 1
per1 = [
    [3, 5, 2, 7, 4, 10, 1, 9, 8, 6],
]

# Permutade choice 2
per2 = [
    [6, 3, 7, 4, 8, 5, 10, 9],
]

# Circular left shift table
shift_table = [1, 2]

# Expansion permutation
dbox = [
    [4, 1, 2, 3, 2, 3, 4, 1],
]

# Transposition permutation
pbox = [
    [2, 4, 3, 1],
]

# Substitution permutation
sbox = [
    [
        [1, 0, 3, 2],
        [3, 2, 1, 0],
        [0, 2, 1, 3],
        [3, 1, 3, 2],
    ],
    [
        [0, 1, 2, 3],
        [2, 0, 1, 3],
        [3, 0, 1, 0],
        [2, 1, 0, 3],
    ],
]

## Cifra de Feistel

### Geração das subchaves
Como a geração das subchaves depende somente da chave original da cifra, podemos modularizar a função `generate_subkeys` do código principal.

In [405]:
# Gera as subchaves a partir da chave principal usando Permuted Choice 1, rotações e Permuted Choice 2
def generate_subkeys(key: Bin):
    pc1 = permuted_choice_1(key) # Aplica PC-1 para reduzir e permutar a chave original
    left, right = halve(pc1)     # Divide a chave em duas metades

    subkeys: List[Bin] = []
    for i in range(2):                                      # Executa duas rodadas (pode ser 16 no DES completo)
        left = circular_left_shift(left, shift_table[i])    # Rotaciona a metade esquerda
        right = circular_left_shift(right, shift_table[i])  # Rotaciona a metade direita
        round_key = permuted_choice_2(left, right)          # Aplica PC-2 para gerar a subchave da rodada
        subkeys.append(round_key)                           # Adiciona subchave à lista

    return subkeys


# Aplica a permutação PC-1 na chave original
def permuted_choice_1(key: Bin):
    return extract_bits(key, per1)


# Junta as metades esquerda e direita e aplica PC-2 para gerar a subchave final da rodada
def permuted_choice_2(left: Bin, right: Bin):
    bits = fuse_bits(left, right)
    return extract_bits(bits, per2)

### Rodadas de Feistel
Segue abaixo o algoritmo `feistel_encrypt` que implementa a cifra de feistel utilizada no DES. O argumento opcional *reverse* presente é utilizado para inverter a ordem das chaves a fim de realizar uma decriptação do ciphertext.

In [406]:
# Realiza a cifra de Feistel sobre o plaintext usando a chave fornecida
def feistel_encrypt(plaintext: Bin, key: Bin, test=False, reverse=False):
    # Gera as subchaves a partir da chave original
    subkeys = generate_subkeys(key)  

    if reverse:  # Inverte as subchaves para decifração
        subkeys.reverse()  

    round_data: List[List[Bin]] = ([])      # Armazena os dados de cada rodada (debug/teste)
    left, right = halve(plaintext)          # Divide o bloco de entrada em duas metades
    for i in range(2):                     
        new_left = right                    # A nova esquerda recebe o valor da direita anterior
        sk = subkeys[i]                     # Subchave da rodada
        magled_right = mangler(right, sk)   # Aplica a função de mistura (mangler)
        new_right = xor(magled_right, left) # XOR da saída da mangler com a esquerda anterior

        left = new_left                      # Atualiza esquerda
        right = new_right                    # Atualiza direita
        round_data.append([left, right, sk]) # Salva os dados da rodada (opcional)

    # Junta esquerda e direita para formar o bloco final
    result = fuse_bits(left, right)  
    return (result, round_data) if test else result  # Retorna dados extras se test=True


# Função de mistura usada em cada rodada da cifra de Feistel
def mangler(bits: Bin, round_subkey: Bin):
    expanded_right = dbox_expansion(bits)           # Expande os bits com D-box
    xored_right = xor(expanded_right, round_subkey) # Aplica XOR com a subchave
    chunks = split_bits(xored_right, 4)             # Divide em blocos de 4 bits
    fused = sbox_substitution(chunks)               # Aplica substituição usando S-box
    result = pbox_permutation(fused)                # Permuta o resultado com P-box
    return result


# Aplica a expansão dos bits de entrada conforme a D-box
def dbox_expansion(bits: Bin):
    return extract_bits(bits, dbox)


# Substitui blocos de 4 bits usando as S-boxes
def sbox_substitution(chunks: List[Bin]):
    result = []
    for round, chunk in enumerate(chunks):
        line = extract_bits(chunk, [[1, 4]])                # Usa bits 1 e 4 para linha
        column = extract_bits(chunk, [[2, 3]])              # Usa bits 2 e 3 para coluna
        sbox_value = sbox[round][line.value][column.value]  # Busca valor na S-box
        result.append(Bin(sbox_value, 2))                   # Valor da S-box tem 2 bits
    return fuse_bits(*result)                               # Junta os blocos substituídos


# Aplica a permutação final usando a P-box
def pbox_permutation(bits: Bin):
    return extract_bits(bits, pbox)

## Simplified Data Encryption Standard
Abaixo segue a função que aplica o S-DES ao plaintext fornecido. É possível notar que os quatro passos descritos na introdução foram usados como base para abstração das subfunções, de forma que eles podem ser facilmente identificados no corpo da função principal.

In [407]:
# Realiza a cifra completa do DES (versão simplificada)
def des_encryption(plaintext: Bin, key: Bin, decrypt=False, test=False):
    ciphertext1 = initial_permutation(plaintext)                     # Aplica a permutação inicial (IP)
    ciphertext2 = feistel_encrypt(ciphertext1, key, reverse=decrypt) # Executa a crifra de Feistel com a chave
    ciphertext3 = swap_halves(ciphertext2)                           # Troca as metades
    ciphertext4 = final_permutation(ciphertext3)                     # Aplica a permutação final (IP⁻¹)

    if test:
        return ciphertext4, [ciphertext1, ciphertext2, ciphertext3]
    
    return ciphertext4


# Aplica a permutação inicial do DES
def initial_permutation(plaintext: Bin):
    return extract_bits(plaintext, initial_perm)


# Aplica a permutação final do DES
def final_permutation(ciphertext: Bin):
    return extract_bits(ciphertext, final_perm)

### Modos de operação
Foram implementados dois modos de operação, o Eletronic Codebook (ECB) e o Cipher Block Chaining (CBC). O modo de operação ECB é o mais simples de todos, ele simplesmente divide o plaintext em blocos de 8 bits e aplica a cifra em cada e concatena os bits em ordem. Por conta dessa simplicidade ele também é considerado inseguro pois ele não utiliza nenhum tipo de aleatoriedade ou encadeamento entre os blocos, o que siginifica que blocos idênticos produzem ciphertexts iguais, o que pode revelar padrões e gerar vulnerabilidades.

Já o modo CBC também funciona de maneira similiar ao dividir o texto em blocos que são utilizados de maneira encadeada sequencial, isto é, o resultado da cifra no bloco *n* é utilizado para realizar um xor com o próximo bloco não cifrado. Essa encadeação faz com que seja necessário um vetor de inicialização para o primeiro bloco da sequência e garante que blocos iguais não produzam as mesmas cifras, entretanto, essa cadeia de dependência impede que técnicas de paralelismo possam ser aplicadas, o que resulta em uma cifra lenta e ineficiente.

In [408]:
# Realiza a cifra DES em modo ECB (Electronic Codebook)
def des_ecb_encrypt(plaintext: Bin, key: Bin):
    chunks = split_bits(plaintext, 8) # Divide o texto em blocos de 8 bits
    ciphertext: List[Bin] = []

    for chunk in chunks: 
        ciphertext.append(des_encryption(chunk, key)) # Aplica a cifra DES em cada bloco

    return fuse_bits(*ciphertext) # Junta os blocos cifrados

# Realiza a cifra DES em modo CBC (Cipher Block Chaining)
def des_cbc_encrypt(plaintext: Bin, key: Bin, iv: Bin):
    chunks = split_bits(plaintext, 8) # Divide o texto em blocos de 8 bits
    ciphertext: List[Bin] = []

    # Inicializa o bloco anterior com o vetor de inicialização (IV)
    previous_chunk = iv 
    for chunk in chunks:
        xor_chunk = xor(chunk, previous_chunk)           # Aplica XOR com o bloco anterior
        encrypted_chunk = des_encryption(xor_chunk, key) # Cifra o bloco resultante
        ciphertext.append(encrypted_chunk)               # Adiciona o bloco cifrado à lista
        previous_chunk = encrypted_chunk                 # Atualiza o bloco anterior para o próximo loop

    return fuse_bits(*ciphertext)

## Testes e exemplos

Durante a implementação foram feitos testes para cada subfunção do algoritmo de DES. Segue abaixo os exemplos e saídas do processamento de cada plaintext (ou ciphertext).

### Teste da Geração de Subchaves
Foi definida uma tabela com uma lista de valores binários esperados para as subchaves geradas de cada um dos rounds de Feistel. Utilizamos uma chave para gerar a subchaves e as comparamos com os valores esperados.

In [409]:
def test_subkey_generation(example_key, subexpected_keys):
    try:
        generated_keys = generate_subkeys(example_key)

    except Exception as e:
        print(f"Test failed with exception: {e}")
        return

    print("\n\nTESTING SUBKEYS GENERATION".center(50))
    print("-" * 60)
    print("EXPECTED KEY   | GENERATED KEY")
    for g1, g2 in zip(subexpected_keys, generated_keys):
        print(f"{g1.to_bin():<14} | {g2.to_bin()}")

    all_match = all(g1 == g2 for g1, g2 in zip(subexpected_keys, generated_keys))
    if all_match:
        print("\nTest passed: All subkeys match expected values.")
    else:
        print("\nTest failed: Subkeys do not match expected values.")

### Teste da Cifra de Feistel

O modo teste da função `feistel_encrypt` retorna os valores da parte esquerda e direita, além da subchave gerada para cada um dos rounds. Esses valores foram comparados com os dados esperados e mostrados em duas tabelas para comparação.

In [410]:
def test_feistel_encrypt(example_plaintext, example_key, expected_rounds):
    try:

        initial_permutation = extract_bits(example_plaintext, initial_perm)
        example_key = Bin(0b1010000010, 10)
        rounds_data = feistel_encrypt(initial_permutation, example_key, test=True)[1]

    except Exception as e:
        print(f"Test failed with exception: {e}")
        return

    print("\n\nTESTING FEISTEL ENCRYPTION".center(50))
    print("-" * 60)

    print("EXPECTED ROUND VALUES")
    print("LEFT       | RIGHT      | SUBKEY")
    for l, r, sk in expected_rounds:
        print(f"{l.to_bin():<10} | {r.to_bin():<10} | {sk.to_bin()}")

    print("\nGENERATED ROUND VALUES")
    print("LEFT       | RIGHT      | SUBKEY")
    for l, r, sk in rounds_data:
        print(f"{l.to_bin():<10} | {r.to_bin():<10} | {sk.to_bin()}")

    print()
    all_match = all(g1 == g2 for g1, g2 in zip(expected_rounds, rounds_data))
    if all_match:
        print("Test passed: All rounds match expected values.")
    else:
        print("Test failed: Rounds do not match expected values.")



### Teste do DES

Foram feitos testes para a criptografia e decriptografia dos valores passados, os quais foram comparados com os valores esperados fornecidos. Além disso, também são mostrados o formato do ciphertext após cada etapa do DES foi aplicada até a geração do ciphertext final. Caso um valor esperado não seja fornecido, o teste pula a verificação para que seja possível visualizar as informações.

In [411]:
def test_des_encryption(
    example_plaintext: Bin,
    example_key: Bin,
    expected_ciphertext: Bin = None,
):
    try:
        generated_ciphertext, test_data = des_encryption(
            example_plaintext, example_key, test=True
        )
    except Exception as e:
        print(f"Test failed with exception: {e}")
        return

    print("\n\nTESTING DES ENCRYPTION".center(50))
    print("-" * 60)

    print(f"{'Key:':<22}{example_key.to_bin()}")
    print(f"{'Plaintext:':<22}{example_plaintext.to_bin(4)}")
    print(f"{'Initial Permutation:':<22}{test_data[0].to_bin(4)}")
    print(f"{'Feistel Rounds:':<22}{test_data[1].to_bin(4)}")
    print(f"{'Swapped:':<22}{test_data[2].to_bin(4)}")
    print(f"{'Obtained Ciphertext:':<22}{generated_ciphertext.to_bin(4)}")

    if expected_ciphertext is None:
        print("Expected ciphertext not provided. Skipping assertion.")
        return
    else:
        print(f"{'Expected Ciphertext:':<22}{expected_ciphertext.to_bin(4)}\n")

        assert (
            generated_ciphertext == expected_ciphertext
        ), "Test failed: Ciphertext does not match expected value."

        print("Test passed: Ciphertext matches expected value.")


def test_des_decryption(
    example_ciphertext: Bin,
    example_key: Bin,
    expected_plaintext: Bin = None,
):
    try:
        generated_plaintext, test_data = des_encryption(
            example_ciphertext, example_key, decrypt=True, test=True
        )

    except Exception as e:
        print(f"Test failed with exception: {e}")
        return

    print("\n\nTESTING DES DECRYPTION".center(50))
    print("-" * 60)

    print(f"{'Key:':<22}{example_key.to_bin()}")
    print(f"{'Ciphertext:':<22}{example_ciphertext.to_bin(4)}")
    print(f"{'Initial Permutation:':<22}{test_data[0].to_bin(4)}")
    print(f"{'Feistel Rounds:':<22}{test_data[1].to_bin(4)}")
    print(f"{'Swapped:':<22}{test_data[2].to_bin(4)}")
    print(f"{'Obtained Plaintext:':<22}{generated_plaintext.to_bin(4)}")

    if expected_plaintext is None:
        print("Expected plaintext not provided. Skipping assertion.")
        return
    else:
        print(f"{'Expected Plaintext:':<22}{expected_plaintext.to_bin(4)}\n")

        assert (
            generated_plaintext == expected_plaintext
        ), "Test failed: Ciphertext does not match expected value."

        print("Test passed: Ciphertext matches expected value.")



### Teste Modos de Operação

Por fim, foram feitos um teste para cada modo de operação e, visto que os valores esperados não foram fornecidos, a verificação foi pulada.

In [412]:

def test_des_ecb_encrypt(
    example_plaintext: Bin, example_key: Bin, expected_ciphertext: Bin = None
):
    try:
        generated_ciphertext = des_ecb_encrypt(example_plaintext, example_key)
    except Exception as e:
        print(f"Test failed with exception: {e}")
        return

    print("\n\nTESTING DES ECB ENCRYPTION".center(50))
    print("-" * 60)

    print(f"{'Key:':<22}{example_key.to_bin()}")
    print(f"{'Plaintext:':<22}{example_plaintext.to_bin(8)}")
    print(f"{'Obtained Ciphertext:':<22}{generated_ciphertext.to_bin(8)}\n")

    if expected_ciphertext is None:
        print("Expected ciphertext not provided. Skipping assertion.")
        return
    else:
        print(f"{'Expected Ciphertext:':<22}{expected_ciphertext.to_bin(8)}")

        assert (
            generated_ciphertext == expected_ciphertext
        ), "Test failed: Ciphertext does not match expected value."

        print("Test passed: Ciphertext matches expected value.")


def test_des_cbc_encrypt(
    example_plaintext: Bin,
    example_key: Bin,
    iv: Bin,
    expected_ciphertext: Bin = None,
):
    try:
        generated_ciphertext = des_cbc_encrypt(example_plaintext, example_key, iv)
    except Exception as e:
        print(f"Test failed with exception: {e}")
        return

    print("\n\nTESTING DES CBC ENCRYPTION".center(50))
    print("-" * 60)

    print(f"{'Key:':<22}{example_key.to_bin()}")
    print(f"{'IV:':<22}{iv.to_bin()}")
    print(f"{'Plaintext:':<22}{example_plaintext.to_bin(8)}")
    print(f"{'Obtained Ciphertext:':<22}{generated_ciphertext.to_bin(8)}\n")

    if expected_ciphertext is None:
        print("Expected ciphertext not provided. Skipping assertion.")
        return
    else:
        print(f"{'Expected Ciphertext:':<22}{expected_ciphertext.to_bin(8)}")

        assert (
            generated_ciphertext == expected_ciphertext
        ), "Test failed: Ciphertext does not match expected value."

        print("Test passed: Ciphertext matches expected value.")

In [413]:
expected_subkeys = [
    Bin(0b10100100, 8),
    Bin(0b01000011, 8),
]
expected_rounds = [
    [Bin(0b1101, 4), Bin(0b1010, 4), Bin(0b10100100, 8)],
    [Bin(0b1010, 4), Bin(0b0010, 4), Bin(0b01000011, 8)],
]

example_plaintext = Bin(0b10010111, 8)
example_key = Bin(0b1010000010, 10)
expected_ciphertext = Bin(0b00111000, 8)

test_subkey_generation(example_key, expected_subkeys)
test_feistel_encrypt(example_plaintext, example_key, expected_rounds)
test_des_encryption(example_plaintext, example_key, expected_ciphertext)
test_des_decryption(expected_ciphertext, example_key, example_plaintext)

example_plaintext2    = Bin(0b11010111_01101100_10111010_11110000, 32)
initialization_vector = Bin(0b01010101, 8)
test_des_ecb_encrypt(example_plaintext2, example_key)
test_des_cbc_encrypt(example_plaintext2, example_key, initialization_vector)

           

TESTING SUBKEYS GENERATION           
------------------------------------------------------------
EXPECTED KEY   | GENERATED KEY
0b10100100     | 0b10100100
0b01000011     | 0b01000011

Test passed: All subkeys match expected values.
           

TESTING FEISTEL ENCRYPTION           
------------------------------------------------------------
EXPECTED ROUND VALUES
LEFT       | RIGHT      | SUBKEY
0b1101     | 0b1010     | 0b10100100
0b1010     | 0b0010     | 0b01000011

GENERATED ROUND VALUES
LEFT       | RIGHT      | SUBKEY
0b1101     | 0b1010     | 0b10100100
0b1010     | 0b0010     | 0b01000011

Test passed: All rounds match expected values.
             

TESTING DES ENCRYPTION             
------------------------------------------------------------
Key:                  0b1010000010
Plaintext:            0b1001_0111
Initial Permutation:  0b0101_1101
Feistel Rounds:       0b1010_0010
Swapped:              0b0010_1010
Obtained Ciphertext:  0b0011_1000
Expected Cipher

## Referências
Sites e guias que foram utilizados para implementação das cifras

https://www.geeksforgeeks.org/data-encryption-standard-des-set-1/

https://www.geeksforgeeks.org/simplified-data-encryption-standard-key-generation/

https://www.geeksforgeeks.org/simplified-data-encryption-standard-set-2/

https://www.geeksforgeeks.org/shift-micro-operations-in-computer-architecture/

https://aprender3.unb.br/pluginfile.php/3076851/mod_resource/content/1/Aula%2008%20-%20Cifra%20de%20Bloco%20-%20DES%20%282%29.pdf